In [1]:
print('hello')


hello


In [107]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

IS_TEST = True


In [124]:
from datetime import datetime, timezone, timedelta
import pytz

PacificTZ = pytz.timezone('America/Los_Angeles')

def calendar_time(t):
    return t.isoformat()

def build_event(id, sev_id):
    attendees = ['mingzhu.liu@robinhood.com']
    now = datetime.utcnow().astimezone(PacificTZ).replace(microsecond=0, second=0, minute=0, hour=13)
    # Find the adjustment for next Thursday
    change = (3 - now.weekday() + 7) % 7 + 7
    
    start_time = now + timedelta(days=change)
    end_time = now + timedelta(days=change, hours=1)
    event = {
      'id': id,
      'summary': '[Test] SEV review',
      # 'location': '800 Howard St., San Francisco, CA 94103',
      'description': f"""
Sev review meeting <ul>
<li><a href="http://go/$sev/{sev_id}">SEV Home link for {sev_id}</a></li>
<li><a href="http://go/$sevreport/{sev_id}">SEV report for {sev_id}</a></li>
</ul>
""",
      'start': {
        'dateTime': calendar_time(start_time),
        'timeZone': str(PacificTZ),
      },
      'end': {
        'dateTime': calendar_time(end_time),
        'timeZone': str(PacificTZ),
      },
      #'recurrence': [
      #   'RRULE:FREQ=DAILY;COUNT=2'
      #],
      'attendees': [
        {'email': attendee} for attendee in attendees
      ],
      'reminders': {
        'useDefault': True,
      },
        'sendUpdates': 'externalOnly',
    }
    return event

In [121]:
from __future__ import print_function

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']


def auth_google():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            secret_path = os.path.expanduser(
                '~/.secret/desktop/client_secret_44897519483-v351hdp73envpd5f6a0i5d3lh7kknisr.apps.googleusercontent.com.json')
            flow = InstalledAppFlow.from_client_secrets_file(
                secret_path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

In [125]:
from __future__ import print_function

from datetime import datetime

def init_cal_svc():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = auth_google()

    service = build('calendar', 'v3', credentials=creds)
    return service

def list_events(service, calendar_id=None):
        # Call the Calendar API
        now = datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        calendar_id = regulate_calendar_id(calendar_id)

        print(f'Getting the upcoming 10 events for {calendar_id}')
        events_result = service.events().list(
            calendarId=calendar_id, timeMin=now,
            maxResults=10, singleEvents=True,
            orderBy='startTime',
        ).execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

        # Prints the start and name of the next 10 events
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['id'], event['summary'])

def regulate_calendar_id(calendar_id):
    if calendar_id is None:
        return 'primary'
    return calendar_id


SEV_REVIEW_CONFERENCE_DATA = {
    'conferenceDataVersion': 1,
    'conferenceData': {
        'entryPoints': [
          {
              'entryPointType': 'video',
              'uri': 'https://meet.google.com/rxa-wapi-mkt',
              'label': 'meet.google.com/rxa-wapi-mkt'
          },
          {'entryPointType': 'more',
          'uri': 'https://tel.meet/rxa-wapi-mkt?pin=4347219653043',
          'pin': '4347219653043'},
          {'regionCode': 'US',
          'entryPointType': 'phone',
          'uri': 'tel:+1-224-420-9951',
          'label': '+1 224-420-9951',
          'pin': '990763978'}
      ],

      'conferenceSolution': {
          'key': {'type': 'hangoutsMeet'},
          'name': 'Google Meet',
          'iconUri': 'https://fonts.gstatic.com/s/i/productlogos/meet_2020q4/v6/web-512dp/logo_meet_2020q4_color_2x_web_512dp.png'
      },
      #  'createRequest': {'requestId': "rxa-wapi-mkt"},
      #'conferenceId': 'rxa-wapi-mkt',
    }
}

def insert_event(service, calendar_id, event):
    ret = service.events().insert(calendarId=regulate_calendar_id(calendar_id), body=event, conferenceDataVersion=1).execute()
    return ret

###### Calendars

def list_calendars(service):
    ret = []
    page_token = None
    while True:
        calendar_list = service.calendarList().list(pageToken=page_token).execute()
        #for calendar_list_entry in calendar_list['items']:
        #    print("%s   -- %s" % (calendar_list_entry['summary'], calendar_list_entry['id']))
        ret += calendar_list['items']
        page_token = calendar_list.get('nextPageToken')
        if not page_token:
            break
    return ret

def find_calendar_by_name(service, name):
    for c in list_calendars(service):
        if c['summary'] == name:
            return c
    return None

def main():
    try:
        svc = init_cal_svc()

        # calendar_id = 'c_kfu1b7vgfcaqr0nb3hcnhdvlnk@group.calendar.google.com'
        calendar = find_calendar_by_name(svc, 'Activation area team calendar')
        calendar_id = calendar['id']
        print(f'calendar_id: {calendar_id}')
        
        # TODO: remove this to insert into team calendar.
        # use this to retriever personal calendar
        calendar_id = None
        
        # Print out existing meetings for double check.
        list_events(svc, calendar_id=calendar_id)

        event_id = 'sevtest12'
        sev_id = 'sev-2022-11-11-sf-api-quota'
        evt = build_event(event_id, sev_id)
        final_evt = {**evt, **SEV_REVIEW_CONFERENCE_DATA}
        #final_evt = evt
        print(final_evt)
        ret = insert_event(svc, calendar_id, final_evt)
        print(ret)
        
        # list_calendars(svc)
    except HttpError as error:
        print('An error occurred: %s' % error)


if __name__ == '__main__':
    main()

calendar_id: c_kfu1b7vgfcaqr0nb3hcnhdvlnk@group.calendar.google.com
Getting the upcoming 10 events for primary
2022-11-14T10:00:00-08:00 6o3jpj1620olnmq62mmckmvfli Fix oncall
2022-11-14T10:30:00-08:00 6486u3u5g28krcqh09bovan0u8 Brhane / Mingzhu
2022-11-14T11:00:00-08:00 1u88hr62cb38501g684ii7k0md_20221114T213000Z Mingzhu / Evan
2022-11-14T13:30:00-08:00 1v4vbjkfdov15q1ir0rc5ta4lo Mingzhu / Shashidhar
2022-11-14T14:00:00-08:00 5vt666e530jof3tovvmaf5iqie Alli / Mingzhu
2022-11-14T14:30:00-08:00 1b9cd8j6191oiirj1ltnn78fks Arun / Mingzhu
2022-11-15T10:00:00-08:00 9a9n9hrrccgf0t9ohe4vval100 ☎ Oncall for Engagement
2022-11-15T10:30:00-08:00 7b0fm4m6ipjso0kngtngp3f996 Mingzhu / Bernard
2022-11-15T11:00:00-08:00 3hd3b1goh2n19nea3nb4okbprv Dat / Mingzhu
2022-11-15T11:30:00-08:00 7ns4fnlel70l06s8mt7jopt7et_20221111T200000Z Monthly Activation All Hands
{'id': 'sevtest12', 'summary': '[Test] SEV review', 'description': '\nSev review meeting <ul>\n<li><a href="http://go/$sev/sev-2022-11-11-sf-api-q

In [62]:
def test_build_event():
    tm = datetime.utcnow().replace(microsecond=0).astimezone(PacificTZ)
    print(tm.isoformat())
    print(str(PacificTZ))
    print(build_event('sev-test-1'))

if IS_TEST:
    test_build_event()

2022-11-11T20:36:59-08:00
America/Los_Angeles
{'id': 'sev-test-1', 'summary': '[Test] SEV review', 'description': 'Sev review meeting <a href="https://google.com"></a>', 'start': {'dateTime': '2022-11-17T20:00:00-08:00', 'timeZone': 'America/Los_Angeles'}, 'end': {'dateTime': '2022-11-17T21:00:00-08:00', 'timeZone': 'America/Los_Angeles'}, 'attendees': [{'email': 'mingzhu.liu@robinhood.com'}], 'reminders': {'useDefault': True}}


In [59]:
from __future__ import print_function


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']


def main():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            secret_path = os.path.expanduser(
                '~/.secret/desktop/client_secret_44897519483-v351hdp73envpd5f6a0i5d3lh7kknisr.apps.googleusercontent.com.json')
            flow = InstalledAppFlow.from_client_secrets_file(
                secret_path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        print('Getting the upcoming 10 events')
        events_result = service.events().list(calendarId='primary', timeMin=now,
                                              maxResults=10, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

        # Prints the start and name of the next 10 events
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])

    except HttpError as error:
        print('An error occurred: %s' % error)


if __name__ == '__main__':
    main()

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [75]:
a = {}
print({**a, **SEV_REVIEW_CONFERENCE_DATA})

{'conferenceData': {'entryPoints': [{'entryPointType': 'video', 'uri': 'https://meet.google.com/rxa-wapi-mkt', 'label': 'meet.google.com/rxa-wapi-mkt'}, {'entryPointType': 'more', 'uri': 'https://tel.meet/rxa-wapi-mkt?pin=4347219653043', 'pin': '4347219653043'}, {'regionCode': 'US', 'entryPointType': 'phone', 'uri': 'tel:+1-224-420-9951', 'label': '+1 224-420-9951', 'pin': '990763978'}], 'conferenceSolution': {'key': {'type': 'hangoutsMeet'}, 'name': 'Google Meet', 'iconUri': 'https://fonts.gstatic.com/s/i/productlogos/meet_2020q4/v6/web-512dp/logo_meet_2020q4_color_2x_web_512dp.png'}, 'conferenceId': 'rxa-wapi-mkt'}}
